In [13]:
import pandas as pd, tensorflow as tf, numpy as np, spacy
from tensorflow import keras
from sklearn.model_selection import GridSearchCV, TimeSeriesSplit, StratifiedKFold
from scikeras.wrappers import KerasRegressor, KerasClassifier
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.regularizers import l2
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report
from keras import metrics
from sklearn.model_selection import train_test_split

In [2]:
headlines_df = pd.read_csv('new_headlines.csv')

headlines = headlines_df.copy()

headlines_df.head()

,date,fortnight,fortnight_day,section,title,diff
0,2010-01-03,0,5,business,China and the other Brics will rebuild a new w...,19.26
1,2010-01-13,1,1,business,US taxpayers gave banks kiss of life Too dange...,-48.18
2,2010-01-27,2,1,business,Tullow takes a tumble as FTSE falters on globa...,-29.37
3,2010-02-10,3,1,business,FTSE moves higher on Greek hopes and Wall Stre...,37.11
4,2010-02-24,4,1,business,FTSE falters ahead of Bernanke testimony HSBC ...,40.37


In [3]:
headlines = headlines_df.copy()

headlines_df.head()

,date,fortnight,fortnight_day,section,title,diff
0,2010-01-03,0,5,business,China and the other Brics will rebuild a new w...,19.26
1,2010-01-13,1,1,business,US taxpayers gave banks kiss of life Too dange...,-48.18
2,2010-01-27,2,1,business,Tullow takes a tumble as FTSE falters on globa...,-29.37
3,2010-02-10,3,1,business,FTSE moves higher on Greek hopes and Wall Stre...,37.11
4,2010-02-24,4,1,business,FTSE falters ahead of Bernanke testimony HSBC ...,40.37


In [4]:
nlp = spacy.load('en_core_web_md')

In [5]:
vectors = headlines_df['title'].apply(lambda x: nlp(x).vector)

vectors

0      [-1.3010186, 0.26826838, -2.0974205, 0.7587583...
1      [-1.1559626, -0.06732634, -2.267186, 0.8647494...
2      [-1.1932443, 0.13201025, -2.1287062, 0.9592638...
3      [-1.2596585, 0.075955436, -1.6795421, 0.978719...
4      [-1.4499966, -0.07736405, -2.2833703, 0.963610...
                             ...                        
381    [-1.155147, 0.10391867, -1.9603742, 0.46184877...
382    [-1.0387702, 0.1456095, -1.8198619, 0.57597536...
383    [-1.3400451, 0.16482167, -2.0387025, 0.3914396...
384    [-1.0087184, -0.122355685, -2.0073137, 0.57646...
385    [-1.0858915, -0.10410232, -2.009698, 0.4340306...
Name: title, Length: 386, dtype: object

In [6]:
vectors_df = pd.DataFrame(vectors.tolist())

headlines = pd.concat([headlines_df.drop(columns=['date', 'title']), vectors_df], axis=1)

headlines['up'] = headlines['diff'].apply(lambda x: 1 if x >= 0 else 0)

headlines = headlines[[column for column in headlines.columns if column != 'diff'] + ['diff']]

headlines.tail()

,fortnight,fortnight_day,section,0,1,2,3,4,5,6,...,292,293,294,295,296,297,298,299,up,diff
381,381,2,business,-1.155147,0.103919,-1.960374,0.461849,3.318084,-0.002882,1.132856,...,-0.585809,-0.061483,-0.736804,1.954284,0.673725,-1.650408,-1.641880,1.828687,1,421.35
382,382,1,business,-1.038770,0.145609,-1.819862,0.575975,3.469416,0.146417,1.204553,...,-0.696766,-0.087798,-1.068300,1.359791,0.661663,-1.281119,-1.825274,1.458141,0,-100.78
383,383,1,business,-1.340045,0.164822,-2.038702,0.391440,3.364496,0.149060,1.374547,...,-0.926733,-0.053264,-0.700962,1.507293,0.631234,-1.209178,-1.860705,2.047713,1,98.19
384,384,1,business,-1.008718,-0.122356,-2.007314,0.576462,3.385915,0.004756,1.106388,...,-0.637391,-0.204573,-0.782671,1.755281,0.670324,-1.571848,-1.692215,1.701142,1,91.28
385,385,1,business,-1.085891,-0.104102,-2.009698,0.434031,3.415793,0.179098,1.359401,...,-0.891932,-0.264010,-0.838934,1.590277,0.565512,-1.467564,-1.595881,2.276219,1,132.93


In [35]:
train, test = train_test_split(headlines, test_size=0.33, shuffle=True, stratify=headlines.up)

X_train, y_train_class, y_train_reg = train.iloc[:, :-2], train['up'], train['diff']
X_test, y_test_class, y_test_reg = test.iloc[:, :-2], test['up'], test['diff']

X_train.columns, X_test.columns = X_train.columns.astype(str), X_test.columns.astype(str)

X_train.head()

,fortnight,fortnight_day,section,0,1,2,3,4,5,6,...,290,291,292,293,294,295,296,297,298,299
78,78,1,business,-0.982327,-0.226930,-1.300690,0.925439,3.226580,-0.350840,1.135700,...,0.706581,-0.591905,-0.403157,0.266019,-1.085677,1.607269,1.201727,-1.275874,-1.627941,1.665863
93,93,1,business,-0.314622,0.130278,-2.053897,1.452581,3.530804,0.224564,1.059770,...,0.376086,-0.180212,0.066082,0.061060,-0.481588,1.118025,1.319803,-1.242505,-1.459419,1.179923
323,323,1,business,-1.207996,-0.186227,-1.849892,0.664106,3.510605,-0.100523,1.163316,...,0.409026,-0.712499,-0.432409,-0.161986,-0.991053,1.471811,0.588430,-1.397541,-1.921136,1.637766
128,128,1,business,-1.189711,0.329115,-1.870780,0.810533,3.276804,0.124171,1.243288,...,0.498406,-0.355467,-0.422054,-0.005834,-1.352194,1.315955,0.835885,-0.992885,-1.786606,1.551305
240,240,1,business,-0.821567,0.187527,-2.203296,0.587267,2.954206,-0.079810,0.957773,...,0.451152,-0.747042,-0.404718,0.172855,-0.770642,1.518124,0.793652,-1.142135,-1.630901,1.980506


In [36]:
colunas_numericas = X_train.select_dtypes(include='number').columns

preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), colunas_numericas),
        ('cat', OneHotEncoder(), ['section'])
    ]
)

pipeline = Pipeline(steps=[('preprocessor', preprocessor)])

X_train = pipeline.fit_transform(X_train)
X_test = pipeline.transform(X_test)

X_train

array([[-1.1319862 , -0.20218512,  0.53045204, ..., -0.39526665,
         1.        ,  0.        ],
       [-0.99276333, -0.20218512,  3.38291475, ..., -2.15301642,
         1.        ,  0.        ],
       [ 1.14198722, -0.20218512, -0.4336177 , ..., -0.49689983,
         1.        ,  0.        ],
       ...,
       [ 1.12342417, -0.20218512,  0.85746895, ..., -0.23322447,
         1.        ,  0.        ],
       [ 1.55037428, -0.20218512,  0.80881944, ...,  0.37935246,
         1.        ,  0.        ],
       [ 0.20455328, -0.20218512,  0.76464816, ..., -0.29100742,
         1.        ,  0.        ]])

In [85]:
def create_model():

    model = keras.models.Sequential([
        keras.layers.Input(shape=[X_train.shape[1]]),
        keras.layers.Dense(100, activation='relu', kernel_regularizer=l2(0.1)),
        keras.layers.Dense(100, activation='relu', kernel_regularizer=l2(0.1)),
        keras.layers.Dense(100, activation='relu', kernel_regularizer=l2(0.1)),
        keras.layers.Dense(100, activation=keras.activations.swish),
        keras.layers.Dense(1, activation='sigmoid')  # Saída com 2 classes e ativação softmax para classificação
    ])

    model.compile(loss='mean_squared_error', metrics=['accuracy'])
    return model

early_stopping = EarlyStopping(monitor='val_loss', patience=2, restore_best_weights=True)

keras_model = KerasClassifier(model=create_model, callbacks=[early_stopping])

param_grid = {
    'optimizer': ['sgd', 'rmsprop', 'adam'],
    'epochs': [50, 100],
}

cv = StratifiedKFold(n_splits=10)

# Configurando o GridSearchCV
grid = GridSearchCV(estimator=keras_model, param_grid=param_grid, cv=cv, scoring='balanced_accuracy', n_jobs=3, verbose=1)

grid_result = grid.fit(X_train, y_train_class)

Fitting 10 folds for each of 6 candidates, totalling 60 fits
Epoch 1/50
Epoch 1/50
Epoch 1/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.5478 - loss: 32.2709
Epoch 2/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.5730 - loss: 32.3304
Epoch 2/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.4557 - loss: 32.5452
Epoch 2/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6417 - loss: 24.7886 
Epoch 3/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6575 - loss: 24.8404 
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6046 - loss: 25.0100 
Epoch 3/50
Epoch 3/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6555 - loss: 20.3514 
Epoch 4/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6192 - loss: 20.4285 
Epoch 4/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6452 - loss: 20.5442 
Epoch 4/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6455 - loss: 16.8436 
Epoch 5/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6279 - loss: 16.91

/home/vitor/anaconda3/lib/python3.12/site-packages/keras/src/callbacks/early_stopping.py:155: UserWarning: Early stopping conditioned on metric `val_loss` which is not available. Available metrics are: accuracy,loss
  current = self.get_monitor_value(logs)
/home/vitor/anaconda3/lib/python3.12/site-packages/keras/src/callbacks/early_stopping.py:155: UserWarning: Early stopping conditioned on metric `val_loss` which is not available. Available metrics are: accuracy,loss
  current = self.get_monitor_value(logs)
/home/vitor/anaconda3/lib/python3.12/site-packages/keras/src/callbacks/early_stopping.py:155: UserWarning: Early stopping conditioned on metric `val_loss` which is not available. Available metrics are: accuracy,loss
  current = self.get_monitor_value(logs)


8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6183 - loss: 17.0202
2/8 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5781 - loss: 14.4629 Epoch 5/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6238 - loss: 13.9069
Epoch 6/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6488 - loss: 13.9592 
Epoch 6/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6037 - loss: 14.0558 
Epoch 6/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6025 - loss: 11.3995 
Epoch 7/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6601 - loss: 11.4489 
Epoch 7/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6311 - loss: 11.5211 
Epoch 7/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6582 - loss: 9.2424 
Epoch 8/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6050 - loss: 9.3077 
Epoch 8/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6493 - loss: 9.3542 
Epoch 8/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5948 - loss: 7.4335 
Epoch 9/50
8/8 ━━━━━━

/home/vitor/anaconda3/lib/python3.12/site-packages/keras/src/callbacks/early_stopping.py:155: UserWarning: Early stopping conditioned on metric `val_loss` which is not available. Available metrics are: accuracy,loss
  current = self.get_monitor_value(logs)
/home/vitor/anaconda3/lib/python3.12/site-packages/keras/src/callbacks/early_stopping.py:155: UserWarning: Early stopping conditioned on metric `val_loss` which is not available. Available metrics are: accuracy,loss
  current = self.get_monitor_value(logs)
/home/vitor/anaconda3/lib/python3.12/site-packages/keras/src/callbacks/early_stopping.py:155: UserWarning: Early stopping conditioned on metric `val_loss` which is not available. Available metrics are: accuracy,loss
  current = self.get_monitor_value(logs)


8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6399 - loss: 11.3504
Epoch 7/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6629 - loss: 16.8343 
Epoch 5/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6641 - loss: 16.82317
Epoch 5/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6077 - loss: 9.2166 
Epoch 8/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6201 - loss: 13.8895 
Epoch 6/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6390 - loss: 13.8878 
Epoch 6/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6324 - loss: 7.3935 
Epoch 9/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6565 - loss: 11.3683 
Epoch 7/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6394 - loss: 11.37993
1/8 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - accuracy: 0.5938 - loss: 6.1916Epoch 7/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6353 - loss: 5.8623 
Epoch 10/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6223 - loss: 9.2251
Epoch 8/50
8/8 ━━━━━━

...

8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6340 - loss: 0.2322 
Epoch 47/100
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6212 - loss: 0.2362 
Epoch 95/100
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6534 - loss: 20.3217 
Epoch 4/100
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6316 - loss: 0.233007
Epoch 48/100
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6141 - loss: 16.8227
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6737 - loss: 0.2214 
Epoch 96/100
Epoch 5/100
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5835 - loss: 0.2470 
Epoch 49/100
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6167 - loss: 0.2369 
Epoch 97/100
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6528 - loss: 13.8713 
Epoch 6/100
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6552 - loss: 0.2263 9
Epoch 50/100
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6171 - loss: 11.3694
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6450 - loss: 0.2293 
Epoch

/home/vitor/anaconda3/lib/python3.12/site-packages/keras/src/callbacks/early_stopping.py:155: UserWarning: Early stopping conditioned on metric `val_loss` which is not available. Available metrics are: accuracy,loss
  current = self.get_monitor_value(logs)


8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6502 - loss: 0.2282 
Epoch 87/100
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6340 - loss: 0.2329 
Epoch 44/100
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6517 - loss: 25.0314 
Epoch 3/100
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6257 - loss: 0.2345 6
Epoch 88/100
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6255 - loss: 0.2349
Epoch 45/100
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6269 - loss: 20.5653
Epoch 4/100
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6141 - loss: 0.2377 
Epoch 89/100
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6361 - loss: 0.2323 
Epoch 46/100
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6471 - loss: 17.0226 
Epoch 5/100
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6502 - loss: 0.2278 
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6759 - loss: 0.2204 
Epoch 47/100
Epoch 90/100
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6229 - loss: 14

/home/vitor/anaconda3/lib/python3.12/site-packages/keras/src/callbacks/early_stopping.py:155: UserWarning: Early stopping conditioned on metric `val_loss` which is not available. Available metrics are: accuracy,loss
  current = self.get_monitor_value(logs)


9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6261 - loss: 7.8124 
Epoch 8/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6242 - loss: 6.0380
Epoch 9/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6143 - loss: 4.5970 
Epoch 10/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6423 - loss: 3.4370
Epoch 11/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6380 - loss: 2.5352
Epoch 12/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.5860 - loss: 1.8593 
Epoch 13/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6231 - loss: 1.3350
Epoch 14/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6209 - loss: 0.9624
Epoch 15/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6560 - loss: 0.6870
Epoch 16/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.6852 - loss: 0.4974
Epoch 17/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6330 - loss: 0.3897
Epoch 18/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6587 - loss: 0.3076
Epoch 19/50
9/9

In [86]:
best = grid_result.best_estimator_

best.fit(X_train, y_train_class)

Epoch 1/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.6258 - loss: 32.0960  
Epoch 2/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6423 - loss: 24.1267 
Epoch 3/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6418 - loss: 19.4113 
Epoch 4/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6185 - loss: 15.6997
Epoch 5/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6186 - loss: 12.6205
Epoch 6/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6372 - loss: 10.0444
Epoch 7/50


/home/vitor/anaconda3/lib/python3.12/site-packages/keras/src/callbacks/early_stopping.py:155: UserWarning: Early stopping conditioned on metric `val_loss` which is not available. Available metrics are: accuracy,loss
  current = self.get_monitor_value(logs)


9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6508 - loss: 7.8837
Epoch 8/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6431 - loss: 6.1036
Epoch 9/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6607 - loss: 4.6494
Epoch 10/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6636 - loss: 3.4842 
Epoch 11/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6149 - loss: 2.5871
Epoch 12/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6148 - loss: 1.8862 
Epoch 13/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6266 - loss: 1.3613
Epoch 14/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6499 - loss: 0.9733
Epoch 15/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6282 - loss: 0.7094
Epoch 16/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6128 - loss: 0.5258
Epoch 17/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6603 - loss: 0.3878
Epoch 18/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6415 - loss: 0.3125
Epoch 19/50
9/9 

KerasClassifier(
	model=<function create_model at 0x7f1950487c40>
	build_fn=None
	warm_start=False
	random_state=None
	optimizer=sgd
	loss=None
	metrics=None
	batch_size=None
	validation_batch_size=None
	verbose=1
	callbacks=[<keras.src.callbacks.early_stopping.EarlyStopping object at 0x7f193a3ba9c0>]
	validation_split=0.0
	shuffle=True
	run_eagerly=False
	epochs=50
	class_weight=None
)

In [87]:
print(classification_report(y_test_class, best.predict(X_test)))

4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
              precision    recall  f1-score   support

           0       0.00      0.00      0.00        47
           1       0.63      1.00      0.78        81

    accuracy                           0.63       128
   macro avg       0.32      0.50      0.39       128
weighted avg       0.40      0.63      0.49       128



/home/vitor/anaconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/vitor/anaconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/vitor/anaconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [14]:
model = keras.models.Sequential([
    keras.layers.Input(shape=[X_train.shape[1]]),
    keras.layers.Dense(250, activation='relu'),
    keras.layers.Dense(250, activation='relu'),
    keras.layers.Dense(250, activation='relu'),
    keras.layers.Dense(1, activation='linear')
])

model.compile(optimizer='adam', 
              loss='mean_squared_error', 
              metrics=[
                metrics.MeanAbsoluteError(),
                metrics.R2Score()
              ])

# Criando o KerasRegressor
keras_model = KerasRegressor(model=model)

In [15]:
param_grid = {
    'optimizer': ['rmsprop', 'sgd'],
    'epochs': [50, 100],
}

# Criando TimeSeriesSplit
tscv = TimeSeriesSplit(n_splits=3)

# Configurando o GridSearchCV
grid = GridSearchCV(estimator=keras_model, param_grid=param_grid, cv=tscv, scoring='neg_mean_squared_error', n_jobs=1)

In [16]:
grid_result = grid.fit(X_train, y_train_reg)

# Mostrando os resultados
print("Melhores parâmetros:", grid_result.best_params_)
print("Melhor pontuação (neg_mean_squared_error):", grid_result.best_score_)

Epoch 1/50


/home/vitor/anaconda3/lib/python3.12/site-packages/keras/src/saving/saving_lib.py:719: UserWarning: Skipping variable loading for optimizer 'adam', because it has 18 variables whereas the saved optimizer has 2 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


166/166 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 1997.8002 - mean_absolute_error: 32.7731 - r2_score: -6.8063e-04
Epoch 2/50
166/166 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 2094.7593 - mean_absolute_error: 33.4216 - r2_score: 0.0280
Epoch 3/50
166/166 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1995.9271 - mean_absolute_error: 32.7756 - r2_score: 0.0745
Epoch 4/50
166/166 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 1782.8607 - mean_absolute_error: 31.2931 - r2_score: 0.1531
Epoch 5/50
166/166 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 1585.8765 - mean_absolute_error: 29.3701 - r2_score: 0.2359
Epoch 6/50
166/166 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1372.2520 - mean_absolute_error: 27.9433 - r2_score: 0.3571
Epoch 7/50
166/166 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1263.0781 - mean_absolute_error: 26.5761 - r2_score: 0.4030
Epoch 8/50
166/166 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1006.9302 - mean_absolute_error: 24.2537 - r2_score: 0.5010
Epoch 9/50
166/166 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms

In [17]:
best = grid_result.best_estimator_

best.fit(X_train, y_train)

NameError: name 'y_train' is not defined

In [ ]:
y_pred = best.predict(X_train)

sum(y_pred - y_train)

663/663 ━━━━━━━━━━━━━━━━━━━━ 0s 483us/step


-21839.88476762771

In [ ]:
y_train 

109     -48.18
110     -48.18
111     -48.18
112     -48.18
113     -48.18
         ...  
21295    72.26
21296    72.26
21297    72.26
21298    72.26
21299    72.26
Name: diff, Length: 21191, dtype: float64